The computations shown here belong to Example 5.2 in the following paper:

[J. Boehm, S. Keicher, Y. Ren: Computing GIT-Fans with Symmetry and the Mori Chamber Decomposition of $\bar{M}_{0,6}$](https://arxiv.org/abs/1603.09241)

In [1]:
using GITFans
using GAP
using Polymake
using Singular

 ┌───────┐   GAP 4.11.0 of 29-Feb-2020
 │  GAP  │   https://www.gap-system.org
 └───────┘   Architecture: x86_64-pc-linux-gnu-julia64-kv7-v1.4
 Configuration:  gmp 6.1.2, Julia GC, Julia 1.4.0, readline
 Loading the library and packages ...
reading ~/.gap/gaprc
 Packages:   GAPDoc 1.6.3, PrimGrp 3.4.0, SmallGrp 1.4.1, TransGrp 2.0.5
 Try '??help' for help. See also '?copyright', '?cite' and '?authors'

Welcome to Nemo version 0.17.2

Nemo comes with absolutely no warranty whatsoever

polymake version 4.0
Copyright (c) 1997-2020
Ewgenij Gawrilow, Michael Joswig, and the polymake team
Technische Universität Berlin, Germany
https://polymake.org

This is free software licensed under GPL; see the source for copying conditions.
There is NO warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



Enter the input as described in Example 5.2 of the paper.

In [2]:
# grading matrix
Q = [
 1  1   0   0   0 ;
 1  0   1   1   0 ;
 1  0   1   0   1 ;
 1  0   0   1   1 ;
 0  1   0   0  -1 ;
 0  1   0  -1   0 ;
 0  1  -1   0   0 ;
 0  0   1   0   0 ;
 0  0   0   1   0 ;
 0  0   0   0   1 ];

# polynomial ring
R, T = Singular.PolynomialRing( Singular.QQ,
           [ "x" * string(i) for i in 1:size( Q, 1 ) ] )

# generators for the ideal
II = Singular.Ideal( R, [
    T[5]*T[10] - T[6]*T[9] + T[7]*T[8],
    T[1]*T[9]  - T[2]*T[7] + T[4]*T[5],
    T[1]*T[8]  - T[2]*T[6] + T[3]*T[5],
    T[1]*T[10] - T[3]*T[7] + T[4]*T[6],
    T[2]*T[10] - T[3]*T[9] + T[4]*T[8],
] )

# symmetries: (2,3)(5,6)(9,10), (1,5,9,10,3)(2,7,8,4,6)
perms_list = [ [1,3,2,4,6,5,7,8,10,9], [5,7,1,6,9,2,8,4,10,3] ];
perms = [ GAP.Globals.PermList(GAP.julia_to_gap(i)) for i in perms_list ];
G = GAP.Globals.Group( GAP.julia_to_gap( perms ) )

GAP: Group([ (2,3)(5,6)(9,10), (1,5,9,10,3)(2,7,8,4,6) ])

Preprocessing:
Encode the GIT fan by the array of orbit cone orbits, the array of homomorphism objects describing the induced actions of G on the orbits, and the matrix Q.

In [3]:
fan_descr = GITFans.GITFan( II, Q, G );

polymake: used package ppl
  The Parma Polyhedra Library ([[wiki:external_software#PPL]]): A C++ library for convex polyhedra
  and other numerical abstractions.
  http://www.cs.unipr.it/ppl/



Execute the fan traversal, that is, compute neighbors of the known cones, and then the smallest representatives in the G-orbits; each cone is encoded by the set of (indices of) maximal cones containing it.
At the same time, also the incidence relation for the orbits is computed.

In [4]:
(hash_list, edges) = GITFans.fan_traversal(fan_descr);

polymake: used package cdd
  cddlib
  Implementation of the double description method of Motzkin et al.
  Copyright by Komei Fukuda.
  http://www-oldurls.inf.ethz.ch/personal/fukudak/cdd_home/



There are six maximal cones, up to G-symmetry.

In [5]:
length(hash_list)

6

We ask Polymake to create the incidence graph of the orbits, and to visualize it.

In [6]:
intergraph = Polymake.graph.graph_from_edges(collect(edges));
Polymake.graph.visual(intergraph)

<!--
polymake for sam
Sat May 9 12:26:01 2020
unnamed
-->


 
 
 unnamed 
 
 

 

 
 
 Transparency 
 
 
 
 
 Rotation 
 
 x-axis 
 y-axis 
 z-axis 
 Reset 
 

 Rotation speed 
 

 


 
 Display 
 
 
 Labels 
 
 


 
 SVG 
 
 
 Download 
 New tab 
 
 Screenshot 
 
 

 <!-- end of settings -->

polymake: used package threejs
   Three.js is a lightweight cross-browser JavaScript library/API used to create and display animated 3D computer graphics on a Web browser.
   See http://github.com/mrdoob for the source code.



We translate the descriptions of the six maximal cones back to cone objects ...

In [7]:
orbit_list = fan_descr[1];
result_cones = map(x -> Polymake.polytope.intersection(
                          GITFans.cones_from_bitlist(orbit_list, x)...), hash_list);

... and expand their G-orbits.

In [8]:
hom = GITFans.action_on_target(Q, G);
expanded = GITFans.orbit_cone_orbits(result_cones, hom);
orbit_lengths = map(length, expanded)

6-element Array{Int64,1}:
  1
 10
 30
 20
 10
  5

There are altogether 76 maximal cones.

In [9]:
sum( orbit_lengths )

76

The full intersection graph of the fan has (76 vertices and) 180 edges.
(A simpleminded visualization of this graph is not very enlightening.)

In [10]:
maxcones = vcat( expanded... );
full_edges = GITFans.edges_intersection_graph(maxcones, 4);
length(full_edges)

180

We create the fan object in Polymake ...

In [11]:
rays_maxcones = [ [ convert( Vector{Rational{BigInt}}, cone.RAYS[i, :] )
                    for i in 1:size( cone.RAYS, 1 ) ]
                  for cone in maxcones ];

allrays = sort( collect( Set( vcat( rays_maxcones... ) ) ) );

index_maxcones = [ sort( [ findfirst( x -> x == v, allrays )-1 for v in rays ] )
                   for rays in rays_maxcones ];
inputrays = hcat( allrays...)';
fanobj = Polymake.fan.PolyhedralFan( INPUT_RAYS = inputrays, INPUT_CONES = index_maxcones )

type: PolyhedralFan<Rational>

INPUT_CONES
	{6 10 11 13 14 15 16 17 18 19}
	{11 12 13 14 16 19}
	{5 6 14 16 18 19}
	{4 6 13 16 17 19}
	{1 6 10 14 15 18}
	{9 10 11 14 16 18}
	{0 6 10 13 15 17}
	{8 10 11 13 16 17}
	{3 6 13 14 15 19}
	{2 6 10 16 17 18}
	…

INPUT_RAYS
	0 0 0 0 1
	0 0 0 1 0
	0 0 1 0 0
	0 1 -1 0 0
	0 1 0 -1 0
	0 1 0 0 -1
	0 1 0 0 0
	1 0 0 1 1
	1 0 1 0 1
	1 0 1 1 0
	…

... and compute its F-vector.

In [12]:
fanobj.F_VECTOR

pm::Vector<pm::Integer>
20 110 240 225 76

We check some statements from the paper.
There are 36 orbit cones of dimension 5, in 4 orbits.

In [13]:
oc = GITFans.orbit_cones( II, Q, G );
hom = GITFans.action_on_target( Q, G );
oco = GITFans.orbit_cone_orbits( oc, hom );
map( length, oco )

4-element Array{Int64,1}:
 10
 15
 10
  1